<a href="https://www.nvidia.com/dli"> <img src="images/nvidia_header.png" style="margin-left: -30px; width: 300px; float: left;"> </a>

# Creating and Running a Custom NeMo Agent Toolkit Workflow

## Agent Types in the NeMo Agent Toolkit

The NeMo Agent Toolkit supports several types of agents, each with different capabilities and use cases:

1. **ReAct Agent**: Based on the Reasoning + Acting paradigm, this agent performs reasoning between tool calls and can use multiple tools to solve complex problems. It follows an iterative thought process of observation, reasoning, action, and feedback.

2. **Tool Calling Agent**: Leverages the native function/tool calling capabilities of modern LLMs. This agent uses tool input schemas to appropriately route requests to the correct tools without requiring explicit reasoning steps between calls.

3. **Reasoning Agent**: Builds on top of an underlying function (like a ReAct or Tool Calling agent) and adds additional reasoning capabilities. This agent type is useful when you need enhanced reasoning on top of another agent's functionality.

4. **Custom Agents**: The NeMo Agent Toolkit is pluggable, so you can create or integrate other agentic workflows

In this notebook, we'll start with a simple agent, and then we'll be working with a **ReAct Agent** to build our math tools workflow.

Let's define a semi-difficult math problem for our agent to attempt to solve.

In [ ]:
%env MATH_PROBLEM="2 + (4 + 1) * (9 - 3 + 4) * 2 + 2 + 3"

(spoiler alert: it's 107)

Now let's load in our environment variables:

In [ ]:
import os

from dotenv import find_dotenv, load_dotenv

load_dotenv(find_dotenv())

api_key = os.getenv("NVIDIA_API_KEY")

## Creating a NeMo Agent Toolkit Workflow

First, we'll create a workflow using existing NeMo Agent Toolkit components. This will be a simple LLM-powered agent that can respond to text.

### 1. Put the Workflow Config Anywhere

Workflow configurations are yaml files that compose the components that the NeMo Agent Toolkit is aware of. They can go anywhere, so let's create a directory to hold the workflow files that we will be working with.

In [ ]:
!mkdir -p workflows

### 2. Create a Workflow Config

Now we'll create a workflow configuration file. This YAML file will define the components of our agent, including:
 
1. The type of the workflow (or, in other words, the type of the agent)
2. The configuration for the workflow
3. Any tools the agent can use
 
The workflow configuration is the composition blueprint for how our agent will function. We'll start with a basic configuration and then enhance it with math capabilities later.

In [ ]:
%%writefile workflows/simple_llm_config.yml

general:
  use_uvloop: true

llms:
  nim_llm:
    _type: nim
    model_name: meta/llama-3.1-70b-instruct
    temperature: 0.0
    max_tokens: 2048

workflow:
  _type: simple_llm_call
  llm_name: nim_llm


Let's break down the configuration file:

- **general**: Contains general settings for the workflow
  - `use_uvloop: true`: Enables uvloop for better async performance

- **llms**: Defines the language models used in the workflow
  - `nim_llm`: A NIM-based LLM configuration
    - `_type: nim`: Specifies that this is a NIM model
    - `model_name: meta/llama-3.1-70b-instruct`: Uses the 70B parameter Llama 3.1 model
    - `temperature: 0.0`: Sets deterministic output (no randomness)

- **workflow**: Defines the agent workflow
  - `_type: simple_llm_call`: Uses a simple agent that makes a single call to an LLM
  - `llm_name: nim_llm`: References the LLM defined above

This configuration creates a simple agent that relies solely on the LLM's pre-trained knowledge to answer questions.

### 3. Run the Workflow

Now let's run our workflow to see it in action. We'll use the `nat workflow run` command, specifying the path to our configuration file.

This command will start the workflow defined in our configuration file. The workflow will use the specified LLM (nim_llm) to process inputs and generate responses.

Even given the semi-complex math equation, the LLM will still likely return the right answer (107).

In [ ]:
!echo "Math Problem: $MATH_PROBLEM"
!nat run --config_file workflows/simple_llm_config.yml --input "Reasoning off. Solve $MATH_PROBLEM"

But... if the equation gets harder, we may go beyond what Llama 3.1 70B can handle.

In [ ]:
%env MATH_PROBLEM="(2 + sqrt(50) + 23^9) mod 13"

This number is 8.07006-ish (obviously).

Let's ask our llm to do this entirely on its own in one shot. I'm so sorry, Llama 3.1 70B.

In [ ]:
!echo "Math Problem: $MATH_PROBLEM"
!nat run --config_file workflows/simple_llm_config.yml --input "$MATH_PROBLEM"

It just gives up and cannot solve this complicated equation. Good try anyway, Llama 3.1 70B. Help is on the way.

## Creating a Custom NeMo Agent Toolkit Workflow

Now we'll create a custom workflow for our math tools agent. The workflow will be a ReAct agent that can perform various mathematical operations that we provide for it as well as doing general reasoning on its own.

## ReAct Agents in the NeMo Agent Toolkit

ReAct agents are based on the [ReAct paper](https://react-lm.github.io/) and follow an iterative thought process to solve problems:

1. **Observation** – The agent receives an input or problem to solve.
2. **Reasoning (Thought)** – The agent thinks about what to do next.
3. **Action** – The agent calls a tool (like a calculator, search API, etc.).
4. **Observation (Feedback)** – The agent examines the tool's response.
5. **Repeat** – If more steps are needed, it repeats the process.

ReAct agents are particularly powerful because they can reason between tool calls and use multiple tools to answer a question, making them flexible for complex tasks.

### 1. Creating New Workflow Components

Now we'll create a new NeMo Agent Toolkit workflow named `math_tools`.

The `--no-install` flag prevents NeMo Agent Toolkit from installing the tool during creation (so we can see how to manually install it later), and the `--workflow-dir` flag specifies where to create the workflow. We'll put it in the workflows directory we created earlier, though we could put it anywhere.

In [ ]:
!nat workflow create --no-install --workflow-dir workflows math_tools

Let's examine what was just created:

In [ ]:
!tree workflows/math_tools

`nat workflow create` has given us a basic library directory with the following:
- boilerplate `pyproject.toml`
- `src/math_tools/` directory with the following files:
    - `__init__.py` - empty initialization file (we will not be using this in this course)
    - `math_tools_function.py` - the file containing the (now-empty) logic for the function component we will write. The boilerplate tool code currently simply parrots back what it is given.
    - `register.py` - NeMo Agent Toolkit entrypoint for defining tools or other extensions.

These files are not very useful initially, but we will build on them during this notebook.

### 2. Updating the Package Configuration

Our first customizations will be to update the `pyproject.toml` file that defines our workflow as an installable Python package. This file specifies:

- Package metadata (name, version, description)
- Dependencies (we'll use `nvidia-nat[langchain]` for ReAct agent support)
- Entry point registration so NeMo Agent Toolkit can discover our components

The entry point maps the `math_tools.register` module to the NeMo Agent Toolkit component system.

In [ ]:
%%writefile workflows/math_tools/pyproject.toml
[build-system]
build-backend = "setuptools.build_meta"
requires = ["setuptools >= 64"]

[project]
name = "math_tools"
version = "0.1.0"
dependencies = [
  "nvidia-nat[langchain]",
]
requires-python = ">=3.12"
description = "NeMo Agent Toolkit workflow for mathematical operations"
classifiers = ["Programming Language :: Python"]


[project.entry-points.'nat.components']
math_tools = "math_tools.register"

Let's look at that `math_tools.register` module:

In [ ]:
from IPython.display import display, Code

with open('workflows/math_tools/src/math_tools/register.py', 'r') as f:
    code_content = f.read()

display(Code(code_content, language='python'))

And let's look at the `math_tools` module that it imports:

In [ ]:
from IPython.display import display, Code

with open('workflows/math_tools/src/math_tools/math_tools_function.py', 'r') as f:
    code_content = f.read()

display(Code(code_content, language='python'))

As you can see, it doesn't do anything yet, but we can install it to ensure that the NeMo Agent Toolkit is able to see our custom components.

### 3. Installing the Workflow

We'll install our workflow package using pip. This makes our custom components available to the NeMo Agent Toolkit.

In [ ]:
%uv pip install -e workflows/math_tools

### 4. Verifying Component Installation

After installation, we should check if our custom components are now visible to the NeMo Agent Toolkit. This confirms that our package was installed correctly and our tools are available for use.

In [ ]:
!nat info components -t function -q math

## Adding Custom Math Tools

Now that we have our basic workflow set up, let's add some custom tools that can perform mathematical operations. We'll create three tools:

1. **calculator_square_root**: Calculates the square root of a number
2. **calculator_modulus**: Calculates the modulo of two numbers
3. **calculator_exponent**: Raises a number to a power

### 1. Understanding Tool Structure in the NeMo Agent Toolkit

Each tool in the NeMo Agent Toolkit follows this pattern:

1. **Configuration Class**: A class that inherits from `FunctionBaseConfig` and provides a tool name
2. **Decorated Function**: A function decorated with `@register_function` that links to the configuration
3. **Implementation**: An inner async function that contains the actual tool logic
4. **Registration**: A `FunctionInfo.from_fn()` call that wraps the function with metadata

In the cell below, we'll implement these tools.

**Your Turn** 🎉🎉🎉

The `calculator_add` function (all the way at the bottom of the cell) below is incomplete - you'll need to finish it using the other tools as examples. Fill in the `calculator_add` function, and then run the cell to write this file to disk.

The solution is provided beneath the cell if you're in a hurry.

In [ ]:
%%writefile workflows/math_tools/src/math_tools/math_tools_function.py
import logging
import math
import re
from typing import List

from nat.builder.builder import Builder
from nat.builder.function_info import FunctionInfo
from nat.cli.register_workflow import register_function
from nat.data_models.function import FunctionBaseConfig

logger = logging.getLogger(__name__)

# Regular expression for matching numbers (integers or decimals)
NUMBER_MATCHER = re.compile(r"\b(0|[1-9]\d*)(\.\d+)?\b")

def parse_numbers(text: str) -> List[str]:
    """Extract all numbers from the input text."""
    return [match.group() for match in NUMBER_MATCHER.finditer(text)]

# Tool Configurations
class SquareRootToolConfig(FunctionBaseConfig, name="calculator_square_root"):
    pass

class ExponentToolConfig(FunctionBaseConfig, name="calculator_exponent"):
    pass

class ModulusToolConfig(FunctionBaseConfig, name="calculator_modulus"):
    pass

class AddToolConfig(FunctionBaseConfig, name="calculator_add"):
    pass

# Tool Implementations
@register_function(config_type=SquareRootToolConfig)
async def calculator_square_root(tool_config: SquareRootToolConfig, builder: Builder):
    async def _calculator_square_root(text: str) -> str:
        numbers = parse_numbers(text)
        if not numbers:
            return "Please provide a number to calculate the square root"
        
        number = float(numbers[0])
        if number < 0:
            return "Cannot calculate square root of a negative number"
            
        result = math.sqrt(number)
        return f"The square root of {number} is {result}"

    yield FunctionInfo.from_fn(
        _calculator_square_root,
        description=("This is a mathematical tool used to calculate the square root of a number. "
                     "It takes a number as input and computes its square root as the output.")
    )

@register_function(config_type=ExponentToolConfig)
async def calculator_exponent(config: ExponentToolConfig, builder: Builder):
    async def _calculator_exponent(text: str) -> str:
        numbers = parse_numbers(text)
        if len(numbers) < 2:
            return "Please provide a base and an exponent"
            
        base = float(numbers[0])
        exponent = float(numbers[1])
        result = base ** exponent
        return f"The result of {base} raised to the power of {exponent} is {result}"

    yield FunctionInfo.from_fn(
        _calculator_exponent,
        description=("This is a mathematical tool used to calculate exponentiation. "
                     "It takes a base and an exponent as input and computes the result of raising the base to the power of the exponent.")
    )

@register_function(config_type=ModulusToolConfig)
async def calculator_modulus(config: ModulusToolConfig, builder: Builder):
    async def _calculator_modulus(text: str) -> str:
        numbers = parse_numbers(text)
        if len(numbers) < 2:
            return "Please provide a dividend and a divisor"
            
        dividend = float(numbers[0])
        divisor = float(numbers[1])
        if divisor == 0:
            return "Cannot perform modulus with zero as divisor"
            
        result = dividend % divisor
        return f"The remainder of {dividend} divided by {divisor} is {result}"

    yield FunctionInfo.from_fn(
        _calculator_modulus,
        description=("This is a mathematical tool used to calculate the remainder of division. "
                     "It takes a dividend and a divisor as input and computes the remainder when the dividend is divided by the divisor.")
    )

@register_function(config_type=AddToolConfig)
async def calculator_add(config: AddToolConfig, builder: Builder):
    async def _calculator_add(text: str) -> str:
        numbers = parse_numbers(text)
        if len(numbers) < 2:
            return "Please provide at least two numbers to add"
            
        nums = [float(num) for num in numbers]
        result = sum(nums)
        if result.is_integer():
            result = int(result)
            
        numbers_str = " + ".join(numbers)
        return f"The sum of {numbers_str} is {result}"

    yield FunctionInfo.from_fn(
        _calculator_add,
        description=("This is a mathematical tool used to add numbers together. "
                     "It takes two or more numbers as input and computes their sum.")
    )

### SOLUTION: Implementation of calculator_add function
 You can copy and paste this into the register.py file cell above
```python

# Start copying here
@register_function(config_type=AddToolConfig)
async def calculator_add(config: AddToolConfig, builder: Builder):
    async def _calculator_add(text: str) -> str:
        numbers = parse_numbers(text)
        if len(numbers) < 2:
            return "Please provide at least two numbers to add"
            
        nums = [float(num) for num in numbers]
        result = sum(nums)
        if result.is_integer():
            result = int(result)
            
        numbers_str = " + ".join(numbers)
        return f"The sum of {numbers_str} is {result}"

    yield FunctionInfo.from_fn(
        _calculator_add,
        description=("This is a mathematical tool used to add numbers together. "
                     "It takes two or more numbers as input and computes their sum.")
    )
# End copying here

```

We will also update the imports in `register.py` so they will be loaded into the NeMo Agent Toolkit when installed:

In [ ]:
%%writefile workflows/math_tools/src/math_tools/register.py
# pylint: disable=unused-import
# flake8: noqa

# Import any tools which need to be automatically registered here
from math_tools.math_tools_function import calculator_square_root
from math_tools.math_tools_function import calculator_exponent
from math_tools.math_tools_function import calculator_modulus
from math_tools.math_tools_function import calculator_add


### 2. Reinstalling After Changes

Now that we've created our custom tools, we need to reinstall the workflow so that the NeMo Agent Toolkit can see the changes.

In [ ]:
%uv pip install -e workflows/math_tools

Let's make sure that the NeMo Agent Toolkit now has knowledge of the new tools:

In [ ]:
!nat info components -t function -q calculator

### 3. Creating the Configuration Directory

Next, we'll create a directory to store our configuration files. As mentioned, this file can be stored anywhere, but we'll put it with our workflow:

In [ ]:
!mkdir -p workflows/math_tools/configs

### 4. Updating the Configuration

Now we'll update our configuration file to include the new tools we've created. We'll add them to the `functions` section and reference them in the `tool_names` list.

In [ ]:
%%writefile workflows/math_tools/configs/config.yml

general:
  use_uvloop: true

functions:
  calculator_exponent:
    _type: calculator_exponent
  calculator_modulus:
    _type: calculator_modulus
  calculator_square_root:
    _type: calculator_square_root
  calculator_add:
    _type: calculator_add

llms:
  nim_llm:
    _type: nim
    model_name: meta/llama-3.1-70b-instruct
    temperature: 0.0

workflow:
  _type: react_agent
  tool_names:
    - calculator_exponent
    - calculator_modulus
    - calculator_square_root
    - calculator_add
  llm_name: nim_llm
  verbose: true


### 5. Testing the Updated Workflow

Now that we've added our custom tools and updated the configuration, let's run the workflow again with a math question. This time, the agent should:

1. Parse the question to understand what's being asked
2. Decide which tools to use (multiplication and possibly addition)
3. Call the appropriate tools with the right inputs
4. Combine the results to provide a final answer

In [ ]:
!echo "Math Problem: $MATH_PROBLEM"
!nat run --config_file workflows/math_tools/configs/config.yml --input "$MATH_PROBLEM"

With a reasonable amount of probability (and a little bit of luck!), our agent should now proudly present the correct answer of around `8.071044921875`.

## Enhancing Our Agent with Built-in Functions

So far, we've created custom tools for our agent. However, the NeMo Agent Toolkit also provides several built-in functions that we can leverage without writing any code. One such function is `current_datetime`, which returns the current date and time.

### 1. The current_datetime Function

This built-in function:
- Returns the current date and time in human redable format
- Can be used to create time-aware agents
- Allows for temporal reasoning in agent responses
- Requires no implementation from us

Let's explore this function:

In [ ]:
# Check if the current_datetime function is available
!nat info components -t function -q current_datetime

Let's update our configuration to include this built-in function. We'll need to:
1. Add the function to the `functions` section of our config
2. Add it to the `tool_names` list in the workflow section

This will make the function available to our agent alongside our custom math tools.

In [ ]:
%%writefile workflows/math_tools/configs/config.yml

general:
  use_uvloop: true

functions:
  calculator_exponent:
    _type: calculator_exponent
  calculator_modulus:
    _type: calculator_modulus
  calculator_square_root:
    _type: calculator_square_root
  calculator_add:
    _type: calculator_add
  current_datetime:
    _type: current_datetime

llms:
  nim_llm:
    _type: nim
    model_name: meta/llama-3.1-70b-instruct
    temperature: 0.0

workflow:
  _type: react_agent
  tool_names:
    - calculator_exponent
    - calculator_modulus
    - calculator_square_root
    - calculator_add
    - current_datetime
  llm_name: nim_llm
  verbose: true


### 2. Testing Combined Capabilities

Let's test our agent with a question that requires both mathematical operations and time awareness. For this test, we'll ask if an exponentiation result is greater than the current hour. The agent should:

1. Get the current time using the `current_datetime` function
2. Extract the hour from the datetime
3. Perform the exponentiation using `calculator_exponent`

In [ ]:
!nat run --config_file workflows/math_tools/configs/config.yml --input "Is 10 mod 3 greater than the current hour?"

## Running the NeMo Agent Toolkit as an API Server

Now that we've tested our agent's capabilities directly, let's explore how to deploy it as an API service that can be accessed by other applications.


### 1. Discovering Telemetry Components

Before we set up observability, let's see what logging and tracing components are available in the NeMo Agent Toolkit:

In [ ]:
!nat info components -t logging
!nat info components -t tracing

### 2. Adding Telemetry to the Configuration

Now let's update our workflow configuration to include telemetry. We'll add console logging (set to WARN level to reduce noise) and Phoenix tracing for observability:

In [ ]:
%%writefile workflows/math_tools/configs/config.yml

general:
  use_uvloop: true
  telemetry:
    logging:
      console:
        _type: console
        level: WARN
    tracing:
      phoenix:
        _type: phoenix
        endpoint: http://localhost:7007/v1/traces
        project: math_tools

functions:
  calculator_exponent:
    _type: calculator_exponent
  calculator_modulus:
    _type: calculator_modulus
  calculator_square_root:
    _type: calculator_square_root
  calculator_add:
    _type: calculator_add
  current_datetime:
    _type: current_datetime

llms:
  nim_llm:
    _type: nim
    model_name: meta/llama-3.1-70b-instruct
    temperature: 0.0

workflow:
  _type: react_agent
  tool_names:
    - calculator_exponent
    - calculator_modulus
    - calculator_square_root
    - calculator_add
    - current_datetime
  llm_name: nim_llm
  verbose: true

### 3. Starting Phoenix for Observability

Let's start the Phoenix server so we can visualize traces from our API calls.

> **Tip**: If the default port is taken, you can run `PHOENIX_PORT=<new_port> phoenix serve` instead.

In [ ]:
!phoenix serve

### 4. Starting the NAT API Server

In a **separate terminal**, start the NeMo Agent Toolkit API server:

```bash
nat serve --config_file workflows/math_tools/configs/config.yml --port 8080
```

You should see:
```
INFO: Started server process [...]
INFO: Application startup complete.
INFO: Uvicorn running on http://0.0.0.0:8080 (Press CTRL+C to quit)
```

Once the server is running, you can send requests to it from this notebook.

### 5. Calling the API

Now that the server is running, let's test it by sending a request. We'll send a math question that requires both calculation and time awareness.

**If you get a connection error, wait a few seconds for the server to finish starting.**

In [ ]:
import requests

response = requests.post(
    url="http://localhost:8080/generate",
    headers={"Content-Type": "application/json"},
    json={"input_message": "Is sqrt(100) less than the current day of the month?"}
)

print(f"Status: {response.status_code}")
if response.text.strip():
    print(f"Response: {response.json()}")

After the request completes, open the Phoenix UI at [http://localhost:7007](http://localhost:7007) to see the execution traces for this API call.

### 6. Stopping the Servers

When you're done testing:

1. **NAT API server**: Press `Ctrl+C` in the terminal where it's running
2. **Phoenix**: Stop the Phoenix process or close the terminal

## Summary

In this notebook, we've created a custom NeMo Agent Toolkit workflow with mathematical tools. We've learned how to:

1. Set up the project structure
2. Create and configure a ReAct agent
3. Implement custom tools for mathematical operations
4. Enhance our agent with built-in functions
5. Deploy our agent as an API service

In the next notebook, we'll explore how to evaluate and profile our agent to ensure it performs well and meets our requirements.